In [1]:
import numpy as np
from time import time


class RNA():

    def __init__(self, rnalist):
        self.rna_list = rnalist
        self.opt_list = None
        self.traceback_list = None


    # Compute the OPT list using Nussino's algorithm
    def opt(self):

        n = len(self.rna_list)
        self.opt_list = np.zeros((n,  n))

        for j in range(5,n):
            for i in range(j-5, -1, -1):        # i should < j-4, the biggest i is j-5
                opt_temp = 0
                pairing = [1 + self.opt_list[i, t-1] + self.opt_list[t+1, j-1] for t in range(i, j-4)\
                   if self.pair_check(self.rna_list[t], self.rna_list[j])]
                if pairing:
                    opt_temp = max(pairing)
                else:
                    pairing =[0]
                # for t in range(i, j-4):
                #     if self.pair_check(self.rna_list[t], self.rna_list[j]):
                #         if t==0:                    # t-1 = -1, so set opt_list[i, t-1] = 0
                #             opt_temp = max(0 + self.opt_list[t+1, j-1] + 1, opt_temp)
                #         else:
                #             opt_temp = max(self.opt_list[i, t-1] + self.opt_list[t+1, j-1] + 1, opt_temp)
                self.opt_list[i, j] = max(self.opt_list[i, j-1], opt_temp)
        return self.opt_list


    def traceback(self):
        # initall the traceback list
        n = len(self.opt_list[0])
        self.traceback_list = np.zeros(n)

        # call the traceback loop to find solution
        self.traceback_list = self.traceback_loop(0, n-1)
        return self.traceback_list


    def traceback_loop(self, i, j):
        opt_num = self.opt_list[i,j]
        if opt_num == 0:
            pass
        elif self.opt_list[i, j] == self.opt_list[i, j-1]:
            self.traceback_list = self.traceback_loop(i, j-1)
        else:
            self.traceback_list[j] = 1

            for t in range(i, j-4):
                if self.pair_check(self.rna_list[t], self.rna_list[j]):
                    # when t=0, t-1 = -1, it out of range, so assume opt_list[i, t-1] = 0
                    if t == 0 and self.opt_list[1, j-1] + 1 ==  self.opt_list[i, j]:
                        self.traceback_list[0] = -1
                        self.traceback_list = self.traceback_loop(1, j-1)
                        break

                    elif self.opt_list[i, t-1] + self.opt_list[t+1, j-1] + 1 ==  self.opt_list[i, j]:
                        self.traceback_list[t] = -1
                        self.traceback_list = self.traceback_loop(t+1, j-1)
                        self.traceback_list = self.traceback_loop(i, t-1)
                        break

        return self.traceback_list


    def pair_check(self, a, b):
        if (a == 'A' and b == 'U') or (a == 'U' and b == 'A') or (a == 'C' and b == 'G') or (a == 'G' and b == 'C'):
            return True

        # for timing random sequence
        elif (a == 0 and b == 1) or (a == 1 and b == 0) or (a == 2 and b == 3) or (a == 3 and b == 2):
            return True

        else:
            return False


# timming test
def timing(plot=False):
    num_list = []
    time_list = [] 
    for n in range(4, 300, 10):
        for i in range(3):
            rna_list = np.random.randint(4, size=n)
            R = RNA(rna_list)

            start_time = time()
            opt_list = R.opt()
            R.traceback()
            end_time = time()

            runtime = 1000 * (end_time - start_time)
            print(n,i,runtime)

            num_list.append(n)
            time_list.append(runtime)
    
    if plot == True:
        import matplotlib.pyplot as plt

        x = np.array(num_list)
        y = np.array(time_list)

        fig,ax = plt.subplots()
        ax.scatter(x, y, label='Nussinov\'s Algorithm')

        z = np.polyfit(x, y, 3)
        p = np.poly1d(z)
        ax.plot(x,p(x),"r--", alpha=0.3)

        n = np.linspace(4,60,num=60//4)
        ax.scatter(n, (n/6)**2, label='$O(n^2)$')
        ax.scatter(n, (n/6)**3, label='$O(n^3)$')

        ax.set_title('Nussinov\'s Algorithm Run Time')
        ax.set_ylabel('Run Time (ms)')
        ax.set_xlabel('Elements Number n')
        ax.legend()

        plt.show()


    return num_list, time_list



def main(timing_test=False, timing_plot=False):
    # Read points from list
    text = list()

    while True:
        try:
            line = input()
        except EOFError:
            break

        line = line.replace(" ", "")

        if line.startswith('#'):
            pass

        rna = line.split()              # will ignal the blank space
        if rna:
            text.append(line) 

    for rna in text:

        rna_list = list(rna)
                
        R = RNA(rna_list)

        start_time = time()
        opt_list = R.opt()
        result_list = R.traceback()
        end_time = time()

        runtime = 1000 * (end_time - start_time)

        # print result
        print(rna)

        if len(result_list) != len(rna_list):
            print('not same')

        for a in result_list:
            if a == 0:
                print('.', end='')
            if a == -1:
                print('(', end='')
            if a == 1:
                print(')', end='')
        print()

        if len(opt_list) <= 15:
            for raw in opt_list:
                for num in raw:
                    print('%d' % num, end=' ')
                print()

        n = len(list(rna))
        print('Summary: %d, %d, %.6f' % (opt_list[0, n-1], n, runtime))
        print()

    
    if timing_test == True and timing_plot == False:
        num_list, time_list = timing()
        return num_list, time_list

    elif timing_plot == True:
        num_list, time_list = timing(plot=1)
        return num_list, time_list



if __name__ == '__main__':
    timing(plot=1)



4 0 0.007867813110351562
4 1 0.0050067901611328125
4 2 0.0030994415283203125
14 0 2.296924591064453
14 1 2.0911693572998047
14 2 2.046823501586914
24 0 16.52693748474121
24 1 16.36815071105957
24 2 16.65973663330078
34 0 55.63783645629883
34 1 54.49080467224121
34 2 54.27384376525879
44 0 132.58600234985352
44 1 128.65400314331055
44 2 128.96323204040527
54 0 250.81419944763184
54 1 245.87178230285645
54 2 249.2969036102295
64 0 429.95405197143555
64 1 427.45304107666016
64 2 430.6769371032715
74 0 685.5080127716064
74 1 680.0918579101562
74 2 682.1370124816895
84 0 1048.792839050293
84 1 1024.2640972137451
84 2 1010.2930068969727
94 0 1433.7780475616455
94 1 1430.5849075317383
94 2 1431.0691356658936
104 0 1973.870038986206
104 1 1963.606834411621
104 2 1961.2300395965576
114 0 2599.0002155303955
114 1 2604.6791076660156
114 2 2609.3688011169434
124 0 3382.8158378601074
124 1 3407.884120941162
124 2 3458.4081172943115
